# Computational Methods for Lineage Reconstruction

**Irepan Salvador-Martínez** 

*Centro Nacional de Análisis Genómico, C/Baldiri Reixac 4, 08028 Barcelona, Spain.*

*irepan_salvador@hotmail.com*

## Abstract

The recent development of genetic lineage recorders, designed to register the genealogical history of cells using induced somatic mutations, has opened the possibility of reconstructing complete animal cell lineages. In order to reconstruct a cell lineage tree from a molecular recorder, it is crucial to use an appropriate reconstruction algorithm. 
Current approaches include algorithms specifically designed for cell lineage reconstruction and the repurposing of phylogenetic algorithms. These methods have, however, the same objective: to uncover the hierarchical relationships between cells and the sequence of cell divisions that have occurred during development. 
In this chapter, I will use the phylogenetic software FastTree to reconstruct a lineage tree, in a step-by-step manner, using data from a simulated CRISPR-Cas9 recorder. To ensure reproducibility,the code is presented as a Jupyter notebook, available (together with the necessary input files) at https://github.com/irepansalvador/lineage_reconstruction_chapter .

**Key words:** Computational methods, Lineage reconstruction, FastTree

## Introduction


In recent years, the development and utilization of genetic lineage recorders have offered a new perspective on studying cell lineages. One of the game-changing tools in this field is CRISPR-based molecular technology. By harnessing the power of CRISPR-Cas9, scientists can introduce targeted genetic modifications into cells, allowing the recording of their lineage relationships through mutations accumulated over time.
The process involves the insertion of a synthetic construct into the genetic material of cells, which can accumulate stochastic mutations upon the induction of CRISPR-Cas9 activity during cell differentiation. These mutations serve as a molecular archive, akin to DNA barcodes, which can be read and deciphered through sequencing techniques. By analyzing the patterns of mutations, researchers can reconstruct the cell lineage, shedding light on the developmental paths taken by individual cells.



The explosion of lineage tracing technologies has also highlighted the importance of using appropriate lineage reconstruction methods.
In some cases, researchers have tailored algorithms specifically designed for cell lineage reconstruction [1, 2], while others have applied phylogenetic reconstruction algorithms to reconstruct cell lineages [3,4].
A recent benchmarking effort conducted via a crowdsourcing innitiative by the DREAM Challenge [5] encouraged participants from diverse fields to submit lineage reconstruction algorithms to reconstruct lineages from different nature and sizes. 
The best performers from this bechmarking study included a distance-based method ("DCLEAR", [1]) and a Machine learning algorithm ("AMberRland", [2]). Interestingly, the widely used phylogenetic reconstruction algorithm FastTree [6] (used as a reference algorithm in the study) performed very well, suggesting it is a reliable and effective tool for cell lineage reconstruction.

In this chapter, I will use FastTree to reconstruct a cell lineage as I consider it has several advantages that make it a suitable choice for cell lineage reconstruction. 
Firstly, its speed and efficiency are noteworthy. With the increasing amount of data generated by lineage tracing technologies, it is crucial to have a reconstruction method that can handle large datasets in a reasonable amount of time. In this sense, FastTree's ability to reconstruct phylogenetic trees from thousands or even millions of sequences quickly and accurately makes it a good candidate.
Additionally, FastTree uses an approximately maximum-likelihood method, a robust and reliable approach for reconstructing phylogenetic trees. Lastly, an important practical advantage is its ease of installation and use, making it accesible to a wide range of researchers with varying levels of computational expertise.

## Requirements

This chapter, including the example code for lineage reconstruction, is written in a Jupyter Notebook with Python as programming language. Jupyter notebooks are documents that combine computer code, plain text, data, rich visualizations like 3D models, charts, graphs and figures, and interactive controls. They provide a fast interactive environment for exploring and visualizing data that can be shared with others and therefore facilitate code reproducibility. 
The Jupyter default programming language (called “kernel” in the Jupyter ecosystem) is Python, although many other languages (e.g. Julia, R, Matlab, Octave) are also supported and can be used by installing their specific kernels. For detailed information of Jupyter notebooks visit https://docs.jupyter.org/en/latest/.

The specific requirements to run this notebook are:

- Jupyter Notebook package
- Python packages: Biopython and Dendropy
- FastTree software
- Input files (FASTA file and transition matrix file)

### Jupyter Notebook

In order to run this notebook it is necessary to install the Jupyter notebook package. In here we will use `pip`, the recommended installation tool for Python. For additional information on the installation of Jupyter please visit the official Jupyter webpage [https://jupyter.org/install](https://jupyter.org/install).

To install Jupyter, run the following command in the terminal: 

`pip install notebook`

Once installed we can open Jupyter notebook with the following command:

`jupyter notebook`

This will open a dashboard in your default browser where you can see the files located in the folder from where you executed the command. If the command is run from the same folder where this notebook is located the dashboard should look similar to the following image.

![Jupyter dashboard]("images/dashboard.png")

<img src="images/dashboard.png" alt="Jupyter dashboard" width="600"/>

Clicking on the on the Jupyter notebook file (".ipynb") will open the notebook on a new browser tab. The notebook can be run cell-by-cell using the interactive buttons on the top bar (see image below). To run a specific cell, first select the cell by clicking on it and then click the "Run" button.

<img src="images/notebook.png" alt="Jupyter notebook" width="600"/>

### Python packages: Biopython and Dendropy

After installing the Jupyter notebook, we need to install the following Python packages:

- Biopython
- DendroPy

#### Biopython

The Biopython Project [7] is an open-source collection of non-commercial Python tools for computational biology and bioinformatics, created by an international association of developers. With Byopython we can parse the most common bioinformatics file formats into Python utilizable data structures. In here, we will use Biopython to parse a FASTA file into a list with sequences and into another list with cell IDs.
Visit the Biopython Tutorial and Cookbook webpage ([http://biopython.org/DIST/docs/tutorial/Tutorial.html](http://biopython.org/DIST/docs/tutorial/Tutorial.html)) to get detailed documentation, including information to get started with Biopython, and specific documentation on a number of modules.


To install Biopython run this command in the terminal:

`pip install biopython`


#### DendroPy 

DendroPy is a Python library for phylogenetic computing. It provides functions for the simulation, processing, and manipulation of phylogenetic trees and character matrices. It also supports the reading and writing of phylogenetic data in a range of formats, such as NEXUS, NEWICK, Phylip, etc.  
In our example we will use Dendropy to visualise the reconstructed lineage tree. It can also be used to compare and summarise trees, to prune tree nodes or to extracte a subtree of interest. A detailed tutorial on how to use the DendroPy library, with annotated practical examples can be found in [https://dendropy.org/primer/](https://dendropy.org/primer/).


To install DendroPy run this command in the terminal:

`pip install DendroPy`



### FastTree software

FastTree is a phylogenetic software that infers approximately-maximum-likelihood phylogenetic trees from alignments of nucleotide or protein sequences. A great advantage of FastTree is that it can handle alignments with up to a million of sequences in a reasonable amount of time and memory.
The easiest way to use FastTree it is to download a binary executable file. Ready to execute binary files for the different OS are provided in [www.microbesonline.org/fasttree/#Install](www.microbesonline.org/fasttree/#Install).

In this example I am using the Linux 64-bit executable (+SSE) double-precision executable binary file ("FastTreeDbl" binary file). The double-precision version is better to resolve very-short branch lengths (i.e. nearly-identical sequences) accurately. For more details on FastTree vistit the webpage [http://www.microbesonline.org/fasttree/](http://www.microbesonline.org/fasttree/) and see the original paper of Price et al [6].

### Input files

#### FASTA file 
For this example I assume sequencing data from a CRISPR-Cas9 lineage recorder is already available and has already passed the relevant quality controls.
To use a phylogenetic reconstruction algorithm like FastTree, the mutations observed in the sequencing reads need to be coded using a character state matrix and formatted in a file that can be recognised by the software. In this example we will use the FASTA format, a text-based format for representing either nucleotide sequences or amino acid (protein) sequences, in which nucleotides or amino acids are represented using single-letter codes.
In general, a FASTA file usually contains multiple sequences, where each sequence is preceded by a header line that starts with the ">" character. This header usually includes the gene ID name, the sequencing read ID, or any other identifier. The next line after the header contains the actual sequence.

Using the following command we can see the first 20 lines of the FASTA file we will use as an example.

In [1]:
!head -20 INPUT/05div_30_targets_20_states.fasta 

>unmutated
000000000000000000000000000000
>cell_001
0000000000-------H0100G0000000
>cell_002
0000000000-------H0100G0000000
>cell_003
000000---------------AG0030000
>cell_004
000000---------------AG0030000
>cell_005
-H0_0-----B4009000D00000000000
>cell_006
-H0_0-----B4009000D0000000D000
>cell_007
-H0_00000000009000D00005020000
>cell_008
-H0_00000000-----------------D
>cell_009
002-----------D000000000-I000D


#### Transition table

A transition (or substitution) matrix describes the frequency at which a character in a nucleotide sequence or a protein sequence changes to other character states over evolutionary time. In case of protein sequences, the scores correspond to a substitution model which that includes also amino-acid stationary frequencies.

According to the FastTree webpage, the file to be used as a transition matrix needs to fulfill the following requirements (for further details visit the FastTree webpage):  

The transition matrix file must be tab-delimited, with the columns A R N D C Q E G H I L K M F P S T W Y V * (in that order). Each entry gives the rate at which one amino acid changes to another. The last column ("*") should have the stationary distribution. Each row must have the row's name as its first field (in order, A R N ... V).
The transition matrix Q and the stationary distribution $\pi$ should satisfy:

- Each amino acid is reachable: each element of $\pi$ must be positive
- The frequencies of the amino acids sum to 1: $\sum \pi_{i} = 1$.
- Each amino acid can mutate away to other amino acids: each element on the diagonal of $Q$ is negative
- Each element off the diagonal of $Q$ is non-negative
- When at the stationary distribution, the frequencies of each amino acid do not change: $Q · \pi = 0$
- The total frequency of all amino acids does not change as the distribution evolves: the sum of each column M is zero
- The average rate of mutation is 1: $\sum Q_{ii} \pi_{i} = -1$
- Evolution is time reversible: $Q_{ij} / Q_{ji} = \pi_{i} / s_{j}$ 


In here we will use a simplified transition matrix, where any state can reach another state with the same probability. Following the requierements of FastTree on the transition table, we will use a transition table that contains a `20x20` matrix, with column and row names corresponding to the 20 amino acid (AA) symbols and values of `0.052631579` for all the cells, except for the diagonal that has values of `-1`. An additional column, named `*`, has values of `0.05` for all rows.
We can look at the transition table file with the following command (for readability, only the first 50 characters of the first 10 lines are shown):

In [2]:
!head -10 INPUT/transition_table.txt | cut -c-50

A	R	N	D	C	Q	E	G	H	I	L	K	M	F	P	S	T	W	Y	V	*
A	-1	0.052631579	0.052631579	0.052631579	0.0526315
R	0.052631579	-1	0.052631579	0.052631579	0.0526315
N	0.052631579	0.052631579	-1	0.052631579	0.0526315
D	0.052631579	0.052631579	0.052631579	-1	0.0526315
C	0.052631579	0.052631579	0.052631579	0.052631579	
Q	0.052631579	0.052631579	0.052631579	0.052631579	
E	0.052631579	0.052631579	0.052631579	0.052631579	
G	0.052631579	0.052631579	0.052631579	0.052631579	
H	0.052631579	0.052631579	0.052631579	0.052631579	


## Methods

In this section I will describe how to reconstruct a lineage tree from sequences coming from a CRISPR-Cas9 recorder. The code used for this example is based on the FastTree reconstruction
example provided by Gong et al [5], available at https://github.com/ofirr/dream_examples .
The code blocks can be executed if this notebook is downloaded and the software requirements are met (see above). To help with code interpretability, before each code block I describe its objective with a brief text. 
Additional comments are also included within the code blocks (for example describing the meaning of a value or a parameter). These comments are always preceded by the `#` character.


### Import packages

As a first step, we will import all the necessary packages to run this example.

In [3]:
import os
import sys
import pandas as pd
import subprocess
import dendropy        # See the Requirements section to install this package 
from Bio import SeqIO  # See the Requirements section to install this package

The following code block will print the version of packages used in this notebook.

In [4]:
import pkg_resources
from IPython.display import display
root_packages = ['biopython', 'dendropy','ipython', 
                 'subprocess','jupyterlab', 'pandas']
root_packages_list = []
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        root_packages_list.append([m.project_name, m.version])
display(pd.DataFrame(root_packages_list,columns=["package", "version"]
                    ).set_index("package").transpose())

package,jupyterlab,ipython,DendroPy,biopython,pandas
version,3.0.3,7.19.0,4.6.1,1.78,1.1.3


### Functions

This is a general function to execute a shell command and return its output as a string. We will use it to run FastTree and retrieve the reconstructed tree as a string (in a NEWICK format).

In [5]:
def run_command(cmd, get_output=False):
    "run command"
    if get_output == False:
        process = subprocess.Popen(cmd)
        process.wait()
    else:
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        # wait for the process to terminate
        stdout, stderr = process.communicate()
        stdout = stdout.decode('UTF-8').strip()
        stderr = stderr.decode('UTF-8').strip()
        return stdout, stderr, process.returncode

### Input and output
Before running FastTree, we will define the INPUT and OUTPUT folders. In this example, the input folder should contain two files (see Requirements section for a description of these files):

-  FASTA file with CRISPR-Cas9 recorder sequences
-  Transition table (necessary for FastTree)


In [6]:
# Define folder for input and output
input_dir = 'INPUT/'
output_dir = "OUTPUT/"

# Input file (without the ".fasta")
name = '05div_30_targets_20_states'
# Define name for output file with the modified sequences using AA symbols
fasta_mod_file  = os.path.join(input_dir, (name + '_for_fasttree.fasta'))
# Define name for output file with the reconstructed tree (in NEWICK format)
newick_output = os.path.join(output_dir, (name + '.newick'))

### Read the input sequences

We will open the FASTA file with the encoded sequences coming from the CRISPR-Cas9 recorder and store the IDs and sequences in the `ids` and `seqs` lists, respectively.

In [7]:
ids  = []
seqs = []

for seq_record in SeqIO.parse(input_dir + name + ".fasta", "fasta"):
    ids.append(seq_record.id)
    seqs.append(str(seq_record.seq))

We can print the first ten values of `ids` and `seqs` lists we just created to make sure they are correct.

In [8]:
for i in range(10):
     print('>{}\t{}\r'.format(ids[i],seqs[i]))

>unmutated	000000000000000000000000000000
>cell_001	0000000000-------H0100G0000000
>cell_002	0000000000-------H0100G0000000
>cell_003	000000---------------AG0030000
>cell_004	000000---------------AG0030000
>cell_005	-H0_0-----B4009000D00000000000
>cell_006	-H0_0-----B4009000D0000000D000
>cell_007	-H0_00000000009000D00005020000
>cell_008	-H0_00000000-----------------D
>cell_009	002-----------D000000000-I000D


**Note:** This example has been generated with a computer simulation implemented as in Salvador-Martínez et al 2019 [3].
Briefly, in this simulation a cell is implemented as a vector of $m=30$ targets. The simulation starts with one cell, with all its targets in an unmutated state. The initial cell then undergoes $d=5$ cell divisions, growing into a population of $N=32$ cells ($N = 2^d$). Following each cell division, each unmutated target can mutate (with a probability $\mu_d = 0.05$) to one of several possible mutated states. Once a target is mutated, it can no longer change, either to revert to the unmutated state or to transit to a new state.
In this example, I have assigned a "0" to an umutated CRISPR-Cas9 target, and the numbers "1-9" and letters "A-J", to the most frequent mutations. If a target is missing due to an inter-target deletion we assign it the character "-".
The code for the simulations is available in the Github repository [https://github.com/irepansalvador/CRISPR_recorders_sims](https://github.com/irepansalvador/CRISPR_recorders_sims).

### Translate characters into amino acid symbols
Then we will translate the encoded mutations into amino-acid characters, so the file can be read by FastTree. We will use a Python "dictionary" to assign a AA character to the encoded mutations in our FASTA file according to an associative array.
First, we create the `symbol_map` dictionary:

In [9]:
symbol_map = {
    '0':'A', '1':'R',
    '2':'N', '3':'D',
    '4':'C', '5':'Q',
    '6':'E', '7':'G',
    '8':'H', '9':'I',
    'A':'L', 'B':'K',
    'C':'M', 'D':'F',
    'E':'P', 'F':'S',
    'G':'T', 'H':'W',
    'I':'Y', 'J':'V', # Here I "merge" two rare characters as
    '_':'V',          # we can only have 20 AA characters 
    '-':'-', # Intertarget deletions are considered missing information
}

Then we use the dictionary to modify each sequence in the `seqs` list. The new AA encoded sequences are stored in the `seqs_trans` list. Then we print out the first ten sequences of the `seqs_trans` list to visually check that they are correct. 

In [10]:
seqs_trans = []
tmp = []
# getting length of list
Nseqs = len(seqs)

# Iterating the sequences
for i in range(Nseqs):
    df = pd.DataFrame(list(seqs[i]))
    df = df[0].map(symbol_map)
    seqs_trans.append(''.join(df.values))

for i in range(10):
     print('>{}\t{}\r'.format(ids[i],seqs_trans[i]))

>unmutated	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
>cell_001	AAAAAAAAAA-------WARAATAAAAAAA
>cell_002	AAAAAAAAAA-------WARAATAAAAAAA
>cell_003	AAAAAA---------------LTAADAAAA
>cell_004	AAAAAA---------------LTAADAAAA
>cell_005	-WAVA-----KCAAIAAAFAAAAAAAAAAA
>cell_006	-WAVA-----KCAAIAAAFAAAAAAAFAAA
>cell_007	-WAVAAAAAAAAAAIAAAFAAAAQANAAAA
>cell_008	-WAVAAAAAAAA-----------------F
>cell_009	AAN-----------FAAAAAAAAA-YAAAF


Finally, we create a new FASTA file with the new AA encoded sequences. This is the file that will be used as input for FastTree later on.

In [11]:
with open(fasta_mod_file, 'w') as f:
    print("writing " + fasta_mod_file)
    for i in range(Nseqs):
        f.write('>{}\r\n{}\r\n'.format(ids[i],seqs_trans[i]))


writing INPUT/05div_30_targets_20_states_for_fasttree.fasta


### Reconstruct the lineage tree and visualise it 

First, we will create a variable with the path to the executable FastTree file and a function to run FastTree as a shell command and retrieve its output in a string variable. The output should be the reconstructed tree in NEWICK format.

In [12]:
fasttree_path = "/home/irepan/software/FastTreeDbl" # Indicate path to executable file 
transition_table = input_dir + "transition_table.txt"

def run_fasttree(newick_output, fasta_mod_file):
    run_command([fasttree_path, 
                 '-pseudo', 
                 '-trans', transition_table,
                 '-out', newick_output, 
                 fasta_mod_file])
    
    reconstructed_tree = dendropy.Tree.get_from_path(
        newick_output,"newick")
    return reconstructed_tree

Then, we use the `run_fasttree` function we just created to reconstruct the lineage tree using the FASTA file as the input. The output, a tree in the NEWICK format, will be stores in the `rec_tree` variable.

In [13]:
rec_tree = run_fasttree(newick_output, fasta_mod_file)

To have a proper representation of the tree, is important that we define the "root" of the tree using the "unmutated" cell included in the FASTA file.

In [14]:
# root the output of FastTree at the leaf node representing the unmutated construct
rec_tree.reroot_at_node(rec_tree.find_node_with_taxon_label('unmutated'), update_bipartitions=False)

<Node object at 0x7f1ac7a55040: 'None' (<Taxon 0x7f1ac7a550a0 'unmutated'>)>

Finally, we can display the reconstructed tree using the `as_ascii_plot()` function of DendroPy. We can define the width of the displayed tree as number of characters. In here we define 80 as the width.

In [15]:
# Alternatively we can also display the branch lengths by uncommenting the following command 
#print(rec_tree.as_ascii_plot(width=80, plot_metric='length'))

print(rec_tree.as_ascii_plot(width=80))

                                                               /------- cell 015
                                                 /-------------+                
                                                 |             \------- cell 016
                                                 |                              
/------------------------------------------------+             /------- cell 001
|                                                |      /------+                
|                                                |      |      \------- cell 002
|                                                \------+                       
|                                                       |      /------- cell 003
|                                                       \------+                
|                                                              \------- cell 004
|                                                                               
|                           

#### Pruning a tree

In case you are interested in looking at the reconstructed tree using only a subset of cells, you can "prune" the reconstructed tree using the cell IDs using the function `extract_tree_with_taxa_labels()` of Dendropy. This option is especially useful when handling trees with hundreds or thousands of cells.
In the following example, we show the topology of a subset tree containing only the first 12 cells.

In [16]:
# Cell IDs for pruning the tree
cells_for_subset_tree = ["cell 001","cell 002","cell 003","cell 004",
                         "cell 005","cell 006","cell 007","cell 008",
                         "cell 009","cell 010","cell 011","cell 012",
                        ]
# tree1 is a subset of the reconstructed tree containing only the first 12 cells
tree1 = rec_tree.extract_tree_with_taxa_labels(labels=cells_for_subset_tree)

# print the subtree topology
print(tree1.as_ascii_plot(width=60))

                                        /---------- cell 001
                              /---------+                   
                              |         \---------- cell 002
/-----------------------------+                             
|                             |         /---------- cell 003
|                             \---------+                   
|                                       \---------- cell 004
|                                                           
+                                       /---------- cell 007
|                             /---------+                   
|                             |         \---------- cell 008
|         /-------------------+                             
|         |                   |         /---------- cell 005
|         |                   \---------+                   
\---------+                             \---------- cell 006
          |                                                 
          |         /---

#### Compare with reference tree

Although is not always possible, in case of having a ground truth tree (or "reference tree") we could quantify the accuracy of our lineage reconstruction using Dendropy. As in our example we are using a simulated tree, we have this possibility. A simple metric that can be used to compare two phylogenetic trees is the Robinson–Foulds distance. In here we will use a scaled implementation of the RF distance, so the RF distance can have minimum value of 0 (of the reconstructed tree is identical to the reference tree) and a maximum value of 1 if none of the bifurcations of the reference tree is found in the reconstructed tree). This provides a measure of the global accuracy of the inferred lineage tree. 

The first step is to read the reference tree file, available as a newick file, creating a Dendropy tree object.

In [17]:
REF_tree = dendropy.Tree.get(
    path=os.path.join(input_dir,'{}_REF.nw'.format(name)),
    schema="newick",
    )

A Dendropy requirement to compare trees is that the trees share a common "TaxonNamespace".

In [18]:
# establish common taxon namespace
tns = dendropy.TaxonNamespace()

# ensure all trees loaded use common namespace
tree1 = dendropy.Tree.get(
        data=REF_tree.as_string(schema="newick"),
        schema='newick', taxon_namespace=tns)
tree2 = dendropy.Tree.get(
        data=rec_tree.as_string(schema="newick"),
        schema='newick', taxon_namespace=tns)


Then we calculate the number of splits (or bipartitions) that difer between the reference and the reconstructed tree using the `dendropy.calculate.treecompare.find_missing_bipartitions()` function. 

In [19]:
# splits in REF tree missing in reconstructed tree
len(dendropy.calculate.treecompare.find_missing_bipartitions(tree1, tree2))

19

We finally divide the number of wrong splits between the total number of splits of the reference tree.

In [20]:
from dendropy.calculate import treecompare

#dendropy.calculate.treecompare.find_missing_bipartitions(tree1, tree2)
split_diff = len(dendropy.calculate.treecompare.find_missing_bipartitions(tree1, tree2))
all_splits = len(tree1.encode_bipartitions())
split_diff / all_splits

0.3064516129032258

Our result is `0.306`, from which we can conclude that 30% of the bifurcations in the reference tree were not found in the reconstructed tree. 

## Conclusion

In conclusion, the use of FastTree as the reconstruction algorithm in this study is justified by its speed, accuracy, and ease of use. Its ability to handle large datasets, estimate evolutionary relationships, and produce reliable results make it a good choice for analyzing the genetic lineage recorders generated through CRISPR-based molecular technology.

However, it is important to highlight that currently there is no consensus on a reconstruction method and further benchmarking of existent and new reconstruction methods is still necessary.
As it was shown in the DREAM challenge, the performance of the reconstruction algorithms depended on the nature and size of the lineage trees to be reconstructed. It is therefore recommended that researchers consider choosing an appropriate reconstruction method based on the features (size, depth, number of targets) of the lineage tree they want to reconstruct.

#### Note about the OS

This notebook has been created with a computer with Ubuntu OS (version 20.04.6) and using Python version 3.8.5, as it can be seen from the output of the following commands:

In [21]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.6 LTS
Release:	20.04
Codename:	focal


In [22]:
!python --version

Python 3.8.5


In Ubuntu, as in many other Linux distributions, Python comes preinstalled. Instructions to install Python in the different OS (Windows, Mac, Linux), as well as detailed documentation, tutorials, and guides can be found on the official webpage [https://www.python.org/](https://www.python.org/).

## References

[1] Wuming Gong, Hyunwoo J Kim, Daniel J Garry, and Il-Youp Kwak. Single cell lineage recon-
struction using distance-based algorithms and the R package, DCLEAR. BMC Bioinformatics,
23(1):103, 2022. ISSN 1471-2105. doi: 10.1186/s12859-022-04633-x.


[2] Alisa Prusokiene, Augustinas Prusokas, and Renata Retkute. Machine learning based lineage
tree reconstruction improved with knowledge of higher level relationships between cells and
genomic barcodes. NAR Genomics and Bioinformatics, 5(3):lqad077, 08 2023. ISSN 2631-9268.
doi: 10.1093/nargab/lqad077. URL https://doi.org/10.1093/nargab/lqad077 .


[3] Irepan Salvador-Martínez, Marco Grillo, Michalis Averof, and Maximilian J Telford. Is it pos-
sible to reconstruct an accurate cell lineage using CRISPR recorders? eLife, 8, jan 2019. ISSN
2050-084X. doi: 10.7554/eLife.40292.


[4] Aaron McKenna, Gregory M Findlay, J. A. Gagnon, M. S. Horwitz, A. F. Schier, and J. Shen-
dure. Whole-organism lineage tracing by combinatorial and cumulative genome editing. Sci-
ence, 353(6298):aaf7907–aaf7907, jul 2016. ISSN 0036-8075. doi: 10.1126/science.aaf7907.


[5] Wuming Gong, Alejandro A. Granados, Jingyuan Hu, Matthew G. Jones, Ofir Raz, Irepan
Salvador-Martínez, Hanrui Zhang, Ke-Huan K. Chow, Il-Youp Kwak, Renata Retkute, Alisa
Prusokiene, Augustinas Prusokas, Alex Khodaverdian, Richard Zhang, Suhas Rao, Robert
Wang, Phil Rennert, Vangala G. Saipradeep, Naveen Sivadasan, Aditya Rao, Thomas Joseph,
Rajgopal Srinivasan, Jiajie Peng, Lu Han, Xuequn Shang, Daniel J. Garry, Thomas Yu, Verena
Chung, Michael Mason, Zhandong Liu, Yuanfang Guan, Nir Yosef, Jay Shendure, Maximil-
ian J. Telford, Ehud Shapiro, Michael B. Elowitz, and Pablo Meyer. Benchmarked approaches
for reconstruction of in vitro cell lineages and in silico models of C. elegans and M. mus-
culus developmental trees. Cell Systems, 12(8):810–826.e4, aug 2021. ISSN 24054712. doi:
10.1016/j.cels.2021.05.008.

[6] Morgan N Price, Paramvir S Dehal, and Adam P Arkin. FastTree 2 - Approximately maximum-
likelihood trees for large alignments. PLoS ONE, 5(3):e9490, mar 2010. ISSN 19326203. doi:
10.1371/journal.pone.0009490.

[7] Peter J. A. Cock, Tiago Antao, Jeffrey T. Chang, Brad A. Chapman, Cymon J. Cox, Andrew
Dalke, Iddo Friedberg, Thomas Hamelryck, Frank Kauff, Bartek Wilczynski, and Michiel
J. L. de Hoon. Biopython: freely available Python tools for computational molecular bi-
ology and bioinformatics. Bioinformatics, 25(11):1422–1423, 03 2009. ISSN 1367-4803. doi:
10.1093/bioinformatics/btp163.